# **Data Cleaning Notebook**

## Objectives
- Evaluate missing data
- Clean data by converting data types, reduces the influence of extreme outliners, log transformation and box-cox / Yeo-Johnson transformation

## Inputs
- outputs/data_collection/house_price_data.csv

## Outputs
- Generate Dataset: outputs/cleaning/house_prive_data_cleaned.csv

## Conclusion
- Data cleaning pipeline
- Drop Variables: ['EnclosedPorch', 'WoodDeckSF', 'BsmtFinType1', 'LotArea', 'BsmtUnfSF', 'BedroomAbvGr', 'BsmtExposure', 'OverallCond']


---

## Change working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [8]:
import os
current_dir = os.getcwd()
current_dir

'/workspaces'

In [6]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

You set a new current directory


In [7]:
current_dir = os.getcwd()
current_dir

'/workspaces'

---

## Get data

Load collected and analysed data from outputs/collection.

In [4]:
import pandas as pd
df_raw_path = "outputs/data_collected/house_pricing_data.csv"
df = pd.read_csv(df_raw_path)
df.head(3)

,1stFlrSF,2ndFlrSF,BedroomAbvGr,BsmtExposure,BsmtFinSF1,BsmtFinType1,BsmtUnfSF,EnclosedPorch,GarageArea,GarageFinish,...,LotFrontage,MasVnrArea,OpenPorchSF,OverallCond,OverallQual,TotalBsmtSF,WoodDeckSF,YearBuilt,YearRemodAdd,SalePrice
0,856,854.0,3.0,No,706,GLQ,150,0.0,548,RFn,...,65.0,196.0,61,5,7,856,0.0,2003,2003,208500
1,1262,0.0,3.0,Gd,978,ALQ,284,NaN,460,RFn,...,80.0,0.0,0,8,6,1262,NaN,1976,1976,181500
2,920,866.0,3.0,Mn,486,GLQ,434,0.0,608,RFn,...,68.0,162.0,42,5,7,920,NaN,2001,2002,223500


---

## Data Cleaning

### Handling Missing Values

There are some missing data. For cleaning it the function DataCleaningEffect() is used (from feature-engine lession)

a. Drop Columns:
If a column has a very high proportion of missing data (like EnclosedPorch and WoodDeckSF), it may be best to drop the column altogether. As well as for LotFrontage (259 missing).

b. Impute Missing Values:
For columns with a moderate amount of missing data, imputation is a good strategy. There are different methods for imputing based on the nature of the data

- For numeric columns (e.g., 2ndFlrSF, BedroomAbvGr, BsmtExposure, GarageYrBlt): You can impute the missing values using the mean, median, or mode (depending on the distribution of the data). The median is often a good choice for columns with skewed distributions or outliers.
- For categorical columns (e.g., BsmtExposure, BsmtFinType1, GarageFinish): Impute the missing values with the mode (most frequent value) since these are categorical variables.
- For columns like MasVnrArea (small number of missing values): Since MasVnrArea has only 8 missing values, impute using the mean or median, or even consider using the mode depending on the column’s nature. If the percentage is very small, drop rows with missing values in some cases.
- For GarageYrBlt (81 missing): An imputation with with the mode or mean of the year values is not useful. Using a more sophisticated method, like predictive modeling or filling based on group statistics (e.g., grouping by the presence of a garage) is better.

c. Fill Missing with Specific Values:
For certain categorical columns, you might want to fill missing values with a specific placeholder like 'Unknown' or 'None' if that's a valid way to handle missing data for that variable.


By using a machine learning model, more sophisticated imputation techniques can be used, such as:

- K-Nearest Neighbors (KNN) imputation: Uses the values of the closest data points to impute missing values.
- Regression-based imputation: You can use a regression model to predict missing values based on other features in the dataset.


Current approach:
- Drop columns with a very high percentage of missing values (e.g., EnclosedPorch, WoodDeckSF)
- Impute missing values for columns with moderate missing data:
  - Numeric columns (e.g., 2ndFlrSF, BedroomAbvGr) using the mean, median, or mode.
  - Categorical columns (e.g., BsmtExposure, BsmtFinType1) using the mode (most frequent value).
-  For small missing data counts (e.g., MasVnrArea), impute with the mean or median.

Considering model-based imputation for more advanced techniques (MERIT)

#### The following varibales will be dropped because the number of missing values is too high

In [ ]:
df = df.drop(columns=['EnclosedPorch', 'WoodDeckSF'])

#### Impute missing values

In [ ]:
# numeric columns:
df['2ndFlrSF'] = df['2ndFlrSF'].fillna(df['2ndFlrSF'].median())

In [ ]:
# categorical columns
df['GarageFinish'] = df['GarageFinish'].fillna(df['GarageFinish'].mode()[0])

In [ ]:
#small number of missing values
df['MasVnrArea'] = df['MasVnrArea'].fillna(df['MasVnrArea'].mean())

### Handling Duplicates (no duplicates has been found)

### Dealing with Incorrect Data Types

In the data inspectation the conclusion was that ['BsmtExposure', 'BsmtFinType1', 'GarageFinish', 'KitchenQual'] have the wrong data type. Here it is adjusted.

In [ ]:
df.info()

In [ ]:
import pandas as pd
import numpy as np

# Step 1: Convert the columns to string type (it will handle NaN as 'nan' string)
df['BsmtExposure'] = df['BsmtExposure'].astype(str)
df['BsmtFinType1'] = df['BsmtFinType1'].astype(str)
df['GarageFinish'] = df['GarageFinish'].astype(str)
df['KitchenQual'] = df['KitchenQual'].astype(str)

# Step 2: Strip any leading/trailing whitespace from the column values
df['BsmtExposure'] = df['BsmtExposure'].str.strip()
df['BsmtFinType1'] = df['BsmtFinType1'].str.strip()
df['GarageFinish'] = df['GarageFinish'].str.strip()
df['KitchenQual'] = df['KitchenQual'].str.strip()

# Step 3: Replace string representation of 'NaN' with 'Unknown'
df['BsmtExposure'] = df['BsmtExposure'].replace("nan", "Unknown")
df['BsmtFinType1'] = df['BsmtFinType1'].replace("nan", "Unknown")
df['GarageFinish'] = df['GarageFinish'].replace("nan", "Unknown")
df['KitchenQual'] = df['KitchenQual'].replace("nan", "Unknown")

# Step 4: Map categorical values to numeric
df['BsmtExposure'] = df['BsmtExposure'].replace({
    "No": 0,
    "Gd": 1,
    "Mn": 2,
    "Av": 3,
    "Ex": 4,
    "Unknown": np.nan  # Replace 'Unknown' with NaN for numerical operations
})

df['BsmtFinType1'] = df['BsmtFinType1'].replace({
    "GLQ": 6,  # Good Living Quarters
    "ALQ": 5,  # Average Living Quarters
    "BLQ": 4,  # Below Average Living Quarters
    "Rec": 3,  # Average Rec Room
    "LwQ": 2,  # Low Quality
    "Unf": 1,  # Unfinished
    "None": 0  # No Basement
})

df['GarageFinish'] = df['GarageFinish'].replace({
    "Fin": 3,  # Finished
    "RFn": 2,  # Rough Finished
    "Unf": 1,  # Unfinished
    "None": 0  # No Garage
})

df['KitchenQual'] = df['KitchenQual'].replace({
    "Ex": 4,  # Excellent
    "Gd": 3,  # Good
    "TA": 2,  # Typical/Average
    "Fa": 1,  # Fair
    "Po": 0   # Poor
})


# Step 5: Check the unique values to confirm that the columns contain only valid values
print("Unique values in BsmtExposure:", df['BsmtExposure'].unique())
print("Unique values in BsmtFinType1:", df['BsmtFinType1'].unique())
print("Unique values in GarageFinish:", df['GarageFinish'].unique())
print("Unique values in KitchenQual:", df['KitchenQual'].unique())

# Step 6: Apply numeric conversion safely (handle non-numeric values)
df['BsmtExposure'] = pd.to_numeric(df['BsmtExposure'], errors='coerce')
df['BsmtFinType1'] = pd.to_numeric(df['BsmtFinType1'], errors='coerce')
df['GarageFinish'] = pd.to_numeric(df['GarageFinish'], errors='coerce')
df['KitchenQual'] = pd.to_numeric(df['KitchenQual'], errors='coerce')

# Step 7: Check the first few rows to ensure the transformation worked
print(df.head())

# Step 8: Calculate the correlation matrix
correlation_matrix = df.corr()
print(correlation_matrix)

### Correcting Outliers (tbd)

### Dealing with reults of the correlation analysis

The following varibales will be dropped because the corrlation values are low and irrelevant for the further process.

In [ ]:
df = df.drop(columns=['BsmtFinType1', 'LotArea', 'BsmtUnfSF', 'BedroomAbvGr', 'BsmtExposure', 'OverallCond'])

## Data Cleaning summary

Find here the data cleaning approaches:

- Drop the following parameters because those have a low impact on the sales price 
['BsmtFinType1', 'LotArea', 'WoodDeckSF', 'BsmtUnfSF', 'BedroomAbvGr', 'BsmtExposure', 'OverallCond', 'EnclosedPorch']

- Drop variables with more then 80% of missing data since these varables wil likely not add much value
['WoodDeckSF', 'EnclosedPorch']

## Push cleaned data to Repo

In [ ]:
import os
try:
  os.makedirs(name='outputs/data_cleaned') # create outputs/data_cleaned folder
except Exception as e:
  print(e)

### Create Test and Train data
Splitting Data: Typically, you split your dataset into train and test sets. For instance:

`A common split is 70% for training and 30% for testing (though this can vary based on the problem and dataset size).

In [ ]:
from sklearn.model_selection import train_test_split

# Split data into train and test sets
TrainSet, TestSet = train_test_split(df, test_size=0.3, random_state=42)

### Train Set

In [ ]:
import pandas as pd
import os

# Ensure output directory exists
os.makedirs("outputs/data_cleaned", exist_ok=True)

# Save the DataFrame to CSV
TrainSet.to_csv("outputs/data_cleaned/TrainSet.csv", index=False)

print(TrainSet.head(10))  # Display the first 10 rows

### Test Set

In [ ]:
import pandas as pd
import os

# Ensure output directory exists
os.makedirs("outputs/data_cleaned", exist_ok=True)

# Save the DataFrame to CSV
TestSet.to_csv("outputs/data_cleaned/TestSet.csv", index=False)

print(TestSet.head(10))  # Display the first 10 rows